In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import cufflinks as cf
cf.set_config_file(offline=True)
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = 'iframe'

# dynamic programing HJB  
* state equation:  
$ dx=\mu dt+\sigma dW_t+u_tdt $  
* loss function:  
$ J_{t \rightarrow N}= \frac{1}{2}(x_N-x_D)^2 + \int_{t}^{N} \frac{1}{2}u_{\tau}^2d\tau $  
* hamiltonian term:  
$ \mathscr{H} = (\mu+u_t)\frac{\partial J}{\partial x}+\frac{1}{2}\sigma^2\frac{\partial^2J}{\partial x^2}+\frac{1}{2}u_t^2 $  
* optimal control:  
$ u_t^* = argmin\{\mathscr{H}\} $  
$ \frac{\partial \mathscr{H}^*}{\partial u_t^*} = 0 $  
$ u_t^* = \frac{\partial J^*}{\partial x} $
* HJB:  
$ 0 = \frac{\partial J^*}{\partial t} + \mathscr{H}^* $  
* take the optimal control back to HJB:  
$ 0 = \frac{\partial J^*}{\partial t}+\mu \frac{\partial J^*}{\partial x} - \frac{1}{2}(\frac{\partial J^*}{\partial x})^2 + \frac{1}{2}\sigma^2\frac{\partial ^2 J^*}{\partial x^2} $  
$ J_N^* = \frac{1}{2}(x_N-x_D)^2 $  
Than try to solve this PDE by finite difference method

In [2]:
def get_u(x):
    return 0.5*(D-x)/dt

def get_u2(n,x,m=0.5):
    k=1
    # return m*(D-x)*(0.0001*n)*k/dt
    return m*(D-x)*n*0.1

def get_u_hjb(opt,x,n):
    if x%1==0:
        return opt.loc[x,n]
    else:
        lower = max(opt.index[opt.index < x])
        higher = min(opt.index[opt.index > x])
        return  opt.loc[higher,n]-(((higher-x)/(higher-lower))*(opt.loc[higher,n]-opt.loc[lower,n]))

# 调整后的普通系统最优控制问题  
deterministic optimal control with HJB solved by explicit FDM  

* state equation:  
$ dx=(D-x_t) dt+\mu dt+u_tdt $
* loss function:  
$ J=\frac{1}{2}(D-x_T)^2+ \int_t^{T}\frac{1}{2}u_s ds $

In [20]:
# 减少插值的点的占比
miu = 0
sigma = 0
dt = 0.01
D = 5
dx = 1


x_cl = -1*(np.arange(0,301)-160)

t_cl=np.arange(0,1/dt+1)*dt

# Initialize the grid with zeros
pannel = np.zeros((len(x_cl),len(t_cl)))

# Subsume the grid points into a dataframe
# with asset price as index and time steps as columns
grid = pd.DataFrame(pannel, index=x_cl, columns=t_cl)

grid.iloc[:,0] = 0.5*((x_cl-D)**2)

opt = pd.DataFrame( np.zeros((len(x_cl),len(t_cl))), index=x_cl, columns=np.arange(0,1/dt+1))

In [21]:
grid

,0.00,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,...,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99,1.00
160,12012.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,11858.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
158,11704.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,11552.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156,11400.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-136,9940.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-137,10082.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-138,10224.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-139,10368.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
opt

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0
160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
print('dt',dt)
print('miu',miu)
# print('sigma',sigma)
for k in range(1, len(t_cl)):
    for i in range(1,len(x_cl)-1):
        # delta = (grid.iloc[i+1,k-1] - grid.iloc[i-1,k-1]) / (2*ds)
        # gamma = (grid.iloc[i+1,k-1]-2*grid.iloc[i,k-1]+grid.iloc[i-1,k-1]) / (ds**2)
        # theta = (-0.5* vol**2 * s[i]**2 * gamma) - (r*s[i]*delta) + (r*grid.iloc[i,k-1])

        Jx = (grid.iloc[i-1,k-1] - grid.iloc[i+1,k-1] )/(2*dx)
        
        grid.iloc[i,k] = grid.iloc[i,k-1] +((D-grid.index[i]+miu)*dt)*Jx  - (Jx**2)*dt/2
        
        opt.iloc[i,len(t_cl)-k] = -Jx

    # Set boundary interpolate
    grid.iloc[0,k] = 2*grid.iloc[1,k] - grid.iloc[2,k] 
    grid.iloc[len(x_cl)-1,k] = 2*grid.iloc[len(x_cl)-2,k] - grid.iloc[len(x_cl)-3,k]
    
    opt.iloc[0,len(t_cl)-k] = 2*opt.iloc[1,len(t_cl)-k] - opt.iloc[2,len(t_cl)-k]
    opt.iloc[len(x_cl)-1,len(t_cl)-k] = 2*opt.iloc[len(x_cl)-2,len(t_cl)-k] - opt.iloc[len(x_cl)-3,len(t_cl)-k]
    
# # Round grid values to 2 decimals
grid = np.around(grid,3)

dt 0.01
miu 0


C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\3481041343.py:12: RuntimeWarning:

overflow encountered in scalar power

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\3481041343.py:17: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\3481041343.py:10: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\3481041343.py:12: RuntimeWarning:

invalid value encountered in scalar add

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\3481041343.py:12: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\3481041343.py:18: RuntimeWarning:

invalid value encountered in scalar subtract



In [24]:
grid.iplot(kind = 'surface', title='optimal loss')

In [31]:
w = np.random.randn(int(1/dt))

new_state= [0]
new_state_hjb = [0]
state = [0]

running_cost = [0]
running_cost_hjb = [0]
# miu=0

for n,i in enumerate(w):
    

    # intuitive method     
    u = get_u2(n,new_state[-1],m=2)

    state.append(state[-1]+(D-state[-1]+miu)*dt+sigma*i)
    
    new_state.append(new_state[-1]+(D-new_state[-1]+miu+u)*dt+sigma*i)
    
    running_cost.append(running_cost[-1]+0.5*(u)**2)
    
    # HJB method    
    u_hjb =1*get_u_hjb(opt,new_state_hjb[-1],n)
    
    # new_state_hjb.append(new_state_hjb[-1]+(D-new_state_hjb[-1]+miu+u_hjb)*dt+sigma*i)

    new_state_hjb.append(new_state_hjb[-1]+(D-new_state_hjb[-1]+miu+u_hjb)*dt+sigma*i)
    
    running_cost_hjb.append(running_cost_hjb[-1]+0.5*(u_hjb)**2)
    
    if (n+1)%100==0:
        print('time_tag',n,'new_state',new_state[-1],'u',u,'hjb_state',new_state_hjb[-1],'u_hjb',u_hjb,)
    
    
fig = go.Figure()

x = range(int(1/dt+1))

fig.add_trace(go.Scatter(x=list(x), y=state, mode='lines',yaxis='y1', name='state'))
fig.add_trace(go.Scatter(x=list(x), y=new_state, mode='lines',yaxis='y1', name='new state'))
fig.add_trace(go.Scatter(x=list(x), y=running_cost, mode='lines',yaxis='y2', name='running_cost'))


# HJB plot
fig.add_trace(go.Scatter(x=list(x), y=new_state_hjb, mode='lines',yaxis='y1', name='new_state_hjb'))
fig.add_trace(go.Scatter(x=list(x), y=running_cost_hjb, mode='lines',yaxis='y2', name='running_cost_hjb'))
    

fig.update_layout(
height=800,
width = 1300,
yaxis1 ={'title':'cost','anchor':'x','side':'left'},
yaxis2 ={'title':'cost','anchor':'x','side':'right','overlaying': 'y1'},
)
fig.show()

print('normal final state: ',new_state[-1],'volatility: ',np.std(new_state),'total_cost',running_cost[-1]+0.5*(D-new_state[-1])**2)
print('hjb final state: ',new_state_hjb[-1],'volatility: ',np.std(new_state_hjb),'total_cost',running_cost_hjb[-1]+0.5*(D-new_state_hjb[-1])**2)
print('state',state[-1],'terminal cost',0.5*(D-state[-1])**2)

time_tag 99 new_state 4.999960624360342 u 0.0009843909914504679 hjb_state 3.7155726555375668 u_hjb 1.2709318822080589


normal final state:  4.999960624360342 volatility:  1.615561871304116 total_cost 1536.5555068488393
hjb final state:  3.7155726555375668 volatility:  1.0725872184260548 total_cost 35.15226399361671
state 3.1698382936338523 terminal cost 1.6747459357245247


# 调整2  
  
* state equation:  
$ dx=x_tdt+\mu dt+u_tdt $
* loss function:  
$ J=\frac{1}{2}(D-x_T)^2+ \int_t^{T}\frac{1}{2}u_s ds $

In [47]:
# 减少插值的点的占比
miu = 0
sigma = 0
dt = 0.01
D = 10
dx = 1


x_cl = -1*(np.arange(0,301)-160)

t_cl=np.arange(0,1/dt+1)*dt

# Initialize the grid with zeros
pannel = np.zeros((len(x_cl),len(t_cl)))

# Subsume the grid points into a dataframe
# with asset price as index and time steps as columns
grid = pd.DataFrame(pannel, index=x_cl, columns=t_cl)

grid.iloc[:,0] = 0.5*((x_cl-D)**2)

opt = pd.DataFrame( np.zeros((len(x_cl),len(t_cl))), index=x_cl, columns=np.arange(0,1/dt+1))

In [48]:
print('dt',dt)
print('miu',miu)
# print('sigma',sigma)
for k in range(1, len(t_cl)):
    for i in range(1,len(x_cl)-1):
        # delta = (grid.iloc[i+1,k-1] - grid.iloc[i-1,k-1]) / (2*ds)
        # gamma = (grid.iloc[i+1,k-1]-2*grid.iloc[i,k-1]+grid.iloc[i-1,k-1]) / (ds**2)
        # theta = (-0.5* vol**2 * s[i]**2 * gamma) - (r*s[i]*delta) + (r*grid.iloc[i,k-1])

        Jx = (grid.iloc[i-1,k-1] - grid.iloc[i+1,k-1] )/(2*dx)
        
        grid.iloc[i,k] = grid.iloc[i,k-1] +((grid.index[i]+miu)*dt)*Jx  - (Jx**2)*dt/2
        
        opt.iloc[i,len(t_cl)-k] = -Jx

    # Set boundary interpolate
    grid.iloc[0,k] = 2*grid.iloc[1,k] - grid.iloc[2,k] 
    grid.iloc[len(x_cl)-1,k] = 2*grid.iloc[len(x_cl)-2,k] - grid.iloc[len(x_cl)-3,k]
    
    opt.iloc[0,len(t_cl)-k] = 2*opt.iloc[1,len(t_cl)-k] - opt.iloc[2,len(t_cl)-k]
    opt.iloc[len(x_cl)-1,len(t_cl)-k] = 2*opt.iloc[len(x_cl)-2,len(t_cl)-k] - opt.iloc[len(x_cl)-3,len(t_cl)-k]
    
# # Round grid values to 2 decimals
grid = np.around(grid,3)

dt 0.01
miu 0


C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\682531411.py:12: RuntimeWarning:

overflow encountered in scalar power

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\682531411.py:18: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\682531411.py:10: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\682531411.py:17: RuntimeWarning:

invalid value encountered in scalar subtract



In [54]:
grid.iplot(kind = 'surface', title='optimal loss')

In [50]:
w = np.random.randn(int(1/dt))

new_state= [0]
new_state_hjb = [0]
state = [0]

running_cost = [0]
running_cost_hjb = [0]
# miu=0

for n,i in enumerate(w):
    

    # intuitive method     
    u = get_u2(n,new_state[-1],m=2)

    state.append(state[-1]+(state[-1]+miu)*dt+sigma*i)
    
    new_state.append(new_state[-1]+(new_state[-1]+miu+u)*dt+sigma*i)
    
    running_cost.append(running_cost[-1]+0.5*(u)**2)
    
    # HJB method    
    u_hjb =1*get_u_hjb(opt,new_state_hjb[-1],n)
    
    # new_state_hjb.append(new_state_hjb[-1]+(D-new_state_hjb[-1]+miu+u_hjb)*dt+sigma*i)

    new_state_hjb.append(new_state_hjb[-1]+(new_state_hjb[-1]+miu+u_hjb)*dt+sigma*i)
    
    running_cost_hjb.append(running_cost_hjb[-1]+0.5*(u_hjb)**2)
    
    if (n+1)%100==0:
        print('time_tag',n,'new_state',new_state[-1],'u',u,'hjb_state',new_state_hjb[-1],'u_hjb',u_hjb,)
    
    
fig = go.Figure()

x = range(int(1/dt+1))

fig.add_trace(go.Scatter(x=list(x), y=state, mode='lines',yaxis='y1', name='state'))
fig.add_trace(go.Scatter(x=list(x), y=new_state, mode='lines',yaxis='y1', name='new state'))
fig.add_trace(go.Scatter(x=list(x), y=running_cost, mode='lines',yaxis='y2', name='running_cost'))


# HJB plot
fig.add_trace(go.Scatter(x=list(x), y=new_state_hjb, mode='lines',yaxis='y1', name='new_state_hjb'))
fig.add_trace(go.Scatter(x=list(x), y=running_cost_hjb, mode='lines',yaxis='y2', name='running_cost_hjb'))
    

fig.update_layout(
height=800,
width = 1300,
yaxis1 ={'title':'cost','anchor':'x','side':'left'},
yaxis2 ={'title':'cost','anchor':'x','side':'right','overlaying': 'y1'},
)
fig.show()

print('normal final state: ',new_state[-1],'volatility: ',np.std(new_state),'total_cost',running_cost[-1]+0.5*(D-new_state[-1])**2)
print('hjb final state: ',new_state_hjb[-1],'volatility: ',np.std(new_state_hjb),'total_cost',running_cost_hjb[-1]+0.5*(D-new_state_hjb[-1])**2)
print('state',state[-1],'terminal cost',0.5*(D-state[-1])**2)

time_tag 99 new_state 10.561412178762378 u -11.251183669328933 hjb_state 7.566990582018573 u_hjb 2.45758527068831


normal final state:  10.561412178762378 volatility:  3.823120944929347 total_cost 10582.43328404532
hjb final state:  7.566990582018573 volatility:  2.1931772037756696 total_cost 935.3472463809073
state 0.0 terminal cost 50.0


# 调整3 减小running cost 权重  
  
* state equation:  
$ dx=x_tdt+\mu dt+u_tdt $
* loss function:  
$ J=\frac{1}{2}(D-x_T)^2+ \int_t^{T}\frac{k}{2}u_s ds $  
$ 0<k<1 $

In [55]:
# 减少插值的点的占比
miu = 0
sigma = 0
dt = 0.01
D = 10
dx = 1
k = 0.8

x_cl = -1*(np.arange(0,301)-160)

t_cl=np.arange(0,1/dt+1)*dt

# Initialize the grid with zeros
pannel = np.zeros((len(x_cl),len(t_cl)))

# Subsume the grid points into a dataframe
# with asset price as index and time steps as columns
grid = pd.DataFrame(pannel, index=x_cl, columns=t_cl)

grid.iloc[:,0] = 0.5*((x_cl-D)**2)

opt = pd.DataFrame( np.zeros((len(x_cl),len(t_cl))), index=x_cl, columns=np.arange(0,1/dt+1))

In [56]:
print('dt',dt)
print('miu',miu)
# print('sigma',sigma)
for k in range(1, len(t_cl)):
    for i in range(1,len(x_cl)-1):
        # delta = (grid.iloc[i+1,k-1] - grid.iloc[i-1,k-1]) / (2*ds)
        # gamma = (grid.iloc[i+1,k-1]-2*grid.iloc[i,k-1]+grid.iloc[i-1,k-1]) / (ds**2)
        # theta = (-0.5* vol**2 * s[i]**2 * gamma) - (r*s[i]*delta) + (r*grid.iloc[i,k-1])

        Jx = (grid.iloc[i-1,k-1] - grid.iloc[i+1,k-1] )/(2*dx)
        
        grid.iloc[i,k] = grid.iloc[i,k-1] +((grid.index[i]+miu)*dt)*Jx  - (Jx**2)*dt*(2-k)/2
        
        opt.iloc[i,len(t_cl)-k] = -Jx

    # Set boundary interpolate
    grid.iloc[0,k] = 2*grid.iloc[1,k] - grid.iloc[2,k] 
    grid.iloc[len(x_cl)-1,k] = 2*grid.iloc[len(x_cl)-2,k] - grid.iloc[len(x_cl)-3,k]
    
    opt.iloc[0,len(t_cl)-k] = 2*opt.iloc[1,len(t_cl)-k] - opt.iloc[2,len(t_cl)-k]
    opt.iloc[len(x_cl)-1,len(t_cl)-k] = 2*opt.iloc[len(x_cl)-2,len(t_cl)-k] - opt.iloc[len(x_cl)-3,len(t_cl)-k]
    
# # Round grid values to 2 decimals
grid = np.around(grid,3)

dt 0.01
miu 0


C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\524257860.py:12: RuntimeWarning:

overflow encountered in scalar power

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\524257860.py:17: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\524257860.py:10: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\524257860.py:18: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\524257860.py:12: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15168\524257860.py:12: RuntimeWarning:

invalid value encountered in scalar add

C:\anaconda3\envs\marketmaking\lib\site-packages\pandas\core\series.py:2602: RuntimeWarning:

overflow encountered in multiply



In [57]:
w = np.random.randn(int(1/dt))

new_state= [0]
new_state_hjb = [0]
state = [0]

running_cost = [0]
running_cost_hjb = [0]
# miu=0

for n,i in enumerate(w):
    

    # intuitive method     
    u = get_u2(n,new_state[-1],m=2)

    state.append(state[-1]+(state[-1]+miu)*dt+sigma*i)
    
    new_state.append(new_state[-1]+(new_state[-1]+miu+u)*dt+sigma*i)
    
    running_cost.append(running_cost[-1]+0.5*(u)**2)
    
    # HJB method    
    u_hjb =1*get_u_hjb(opt,new_state_hjb[-1],n)
    
    # new_state_hjb.append(new_state_hjb[-1]+(D-new_state_hjb[-1]+miu+u_hjb)*dt+sigma*i)

    new_state_hjb.append(new_state_hjb[-1]+(new_state_hjb[-1]+miu+u_hjb)*dt+sigma*i)
    
    running_cost_hjb.append(running_cost_hjb[-1]+0.5*(u_hjb)**2)
    
    if (n+1)%100==0:
        print('time_tag',n,'new_state',new_state[-1],'u',u,'hjb_state',new_state_hjb[-1],'u_hjb',u_hjb,)
    
    
fig = go.Figure()

x = range(int(1/dt+1))

fig.add_trace(go.Scatter(x=list(x), y=state, mode='lines',yaxis='y1', name='state'))
fig.add_trace(go.Scatter(x=list(x), y=new_state, mode='lines',yaxis='y1', name='new state'))
fig.add_trace(go.Scatter(x=list(x), y=running_cost, mode='lines',yaxis='y2', name='running_cost'))


# HJB plot
fig.add_trace(go.Scatter(x=list(x), y=new_state_hjb, mode='lines',yaxis='y1', name='new_state_hjb'))
fig.add_trace(go.Scatter(x=list(x), y=running_cost_hjb, mode='lines',yaxis='y2', name='running_cost_hjb'))
    

fig.update_layout(
height=800,
width = 1300,
yaxis1 ={'title':'cost','anchor':'x','side':'left'},
yaxis2 ={'title':'cost','anchor':'x','side':'right','overlaying': 'y1'},
)
fig.show()

print('normal final state: ',new_state[-1],'volatility: ',np.std(new_state),'total_cost',running_cost[-1]+0.5*(D-new_state[-1])**2)
print('hjb final state: ',new_state_hjb[-1],'volatility: ',np.std(new_state_hjb),'total_cost',running_cost_hjb[-1]+0.5*(D-new_state_hjb[-1])**2)
print('state',state[-1],'terminal cost',0.5*(D-state[-1])**2)

ValueError: max() arg is an empty sequence